# Query data from sakila (MySQL sample database)

## 1. Open connection to database

In [1]:
import mysql.connector as mysql

# use mysql sakila database
db = mysql.connect(
    user='root',
    password='1',
    database='sakila',
    port='1703',
    host='localhost'
)

cursor = db.cursor(buffered=True)

In [2]:
# tabulate is used to print the data in table format
from tabulate import tabulate

## 2. Write all table schema into txt file

In [3]:
cursor.execute("SHOW TABLES")
result = cursor.fetchall()

table_list = [i[0] for i in result]

with open("schema.txt","w") as f:
    for table_name in table_list:
        query = f"EXPLAIN {table_name}"
        cursor.execute(query)
        result = cursor.fetchall()

        headers = [i[0] for i in cursor.description]
        
        f.write(f"Table: {table_name}\n")
        f.write(tabulate(result,headers))
        f.write("\n\n\n")

## 4. SELECT queries

In [4]:
query = (
    "SELECT actor_id, first_name, last_name FROM actor " 
    "WHERE actor_id > 12 AND actor_id < 30 "
    "ORDER BY first_name, last_name DESC "
    "LIMIT 10 "
)
cursor.execute(query)
result = cursor.fetchall()

headers = [i[0] for i in cursor.description]

print(tabulate(result,headers))

  actor_id  first_name    last_name
----------  ------------  -----------
        29  ALEC          WAYNE
        19  BOB           FAWCETT
        24  CAMERON       STREEP
        15  CUBA          OLIVIER
        18  DAN           TORN
        22  ELVIS         MARX
        16  FRED          COSTNER
        17  HELEN         VOIGHT
        27  JULIA         MCQUEEN
        25  KEVIN         BLOOM


In [5]:
query = (
    "SELECT a.first_name, a.last_name, f.title "
    "FROM actor a "
    "JOIN film_actor fa ON a.actor_id = fa.actor_id "
    "JOIN film f ON f.film_id = fa.film_id "
    "ORDER BY a.first_name, a.last_name, f.title ASC "
    "LIMIT 10 "
) # LIST OF ACTORS AND FILMS THEY STARRED IN
cursor.execute(query)
result = cursor.fetchall()
result = [(f"{i[0]} {i[1]}",i[2]) for i in result]

headers = [i[0] for i in cursor.description]
headers = ["name",headers[2]]

print(tabulate(result,headers))

name        title
----------  ---------------------
ADAM GRANT  ANNIE IDENTITY
ADAM GRANT  BALLROOM MOCKINGBIRD
ADAM GRANT  DISCIPLE MOTHER
ADAM GRANT  FIREBALL PHILADELPHIA
ADAM GRANT  GLADIATOR WESTWARD
ADAM GRANT  GLORY TRACY
ADAM GRANT  GROUNDHOG UNCUT
ADAM GRANT  HAPPINESS UNITED
ADAM GRANT  IDOLS SNATCHERS
ADAM GRANT  LOSER HUSTLER


In [6]:
query = (
    "SELECT a.first_name, a.last_name, COUNT(f.film_id) film_count "
    "FROM actor a "
    "JOIN film_actor fa ON a.actor_id = fa.actor_id "
    "JOIN film f ON f.film_id = fa.film_id "
    "GROUP BY a.actor_id, a.first_name, a.last_name "
    "LIMIT 10"
) # LIST OF ACTORS AND NUMBER OF FILMS THEY STARRED IN
cursor.execute(query) 
result = cursor.fetchall()
result = [(f"{i[0]} {i[1]}",i[2]) for i in result]

headers = [i[0] for i in cursor.description]
headers = ["name",headers[2]]

print(tabulate(result,headers))

name                   film_count
-------------------  ------------
PENELOPE GUINESS               19
NICK WAHLBERG                  25
ED CHASE                       22
JENNIFER DAVIS                 22
JOHNNY LOLLOBRIGIDA            29
BETTE NICHOLSON                20
GRACE MOSTEL                   30
MATTHEW JOHANSSON              20
JOE SWANK                      25
CHRISTIAN GABLE                22


In [7]:
query = (
    "WITH "
        "actor_film_count AS "
        "( "
        "SELECT a.actor_id, COUNT(f.film_id) film_count "
        "FROM actor a "
        "JOIN film_actor fa ON a.actor_id = fa.actor_id "
        "JOIN film f ON f.film_id = fa.film_id "
        "GROUP BY a.actor_id "
        ") "
    "SELECT a.first_name, a.last_name, afc.film_count "
    "FROM actor a "
    "JOIN actor_film_count afc ON a.actor_id = afc.actor_id "
    "LIMIT 10 "
) # LIST OF ACTORS AND NUMBER OF FILMS THEY STARRED IN
cursor.execute(query)
result = cursor.fetchall()
result = [(f"{i[0]} {i[1]}",i[2]) for i in result]

headers = [i[0] for i in cursor.description]
headers = ["name",headers[2]]

print(tabulate(result,headers))

name                   film_count
-------------------  ------------
PENELOPE GUINESS               19
NICK WAHLBERG                  25
ED CHASE                       22
JENNIFER DAVIS                 22
JOHNNY LOLLOBRIGIDA            29
BETTE NICHOLSON                20
GRACE MOSTEL                   30
MATTHEW JOHANSSON              20
JOE SWANK                      25
CHRISTIAN GABLE                22


In [8]:
query = (
    "SELECT a.first_name, a.last_name, COUNT(f.film_id) film_count "
    "FROM actor a "
    "JOIN film_actor fa ON a.actor_id = fa.actor_id "
    "JOIN film f ON f.film_id = fa.film_id "
    "GROUP BY a.actor_id, a.first_name, a.last_name "
    "HAVING film_count > 25 "
    "LIMIT 10 "
) # LIST OF ACTORS AND NUMBER OF FILMS THEY STARRED IN (MORE THAN 25 FILMS ONLY)
cursor.execute(query) 
result = cursor.fetchall()
result = [(f"{i[0]} {i[1]}",i[2]) for i in result]

headers = [i[0] for i in cursor.description]
headers = ["name",headers[2]]

print(tabulate(result,headers))

name                   film_count
-------------------  ------------
JOHNNY LOLLOBRIGIDA            29
GRACE MOSTEL                   30
KARL BERRY                     31
UMA WOOD                       35
VIVIEN BERGEN                  30
CUBA OLIVIER                   28
FRED COSTNER                   27
HELEN VOIGHT                   32
LUCILLE TRACY                  30
KIRSTEN PALTROW                27


## 5. INSERT, UPDATE, DELETE queries

In [9]:
# SETUP AUTO_INCREMENT PRIMARY KEY
largest_id_query = (
    "SELECT MAX(actor_id) FROM actor "
)
cursor.execute(largest_id_query)
largest_id = cursor.fetchone()[0]
print(f"Largest ID: {largest_id}")

alter_query = (
    "ALTER TABLE actor AUTO_INCREMENT = %s"
)
cursor.execute(alter_query,(largest_id+1,))

Largest ID: 200


In [10]:
# INSERT NEW ROW
insert_query = (
    "INSERT INTO actor (first_name,last_name)"
    "VALUES"
        "('KHANH','BUI')"
)
cursor.execute(insert_query)
db.commit()

# CHECK LAST 5 ROWS
check_query_1 = (
    "SELECT * FROM actor "
    "ORDER BY actor_id DESC "
    "LIMIT 5 "
)
cursor.execute(check_query_1)
result = cursor.fetchall()
headers = [i[0] for i in cursor.description]
print("LAST 5 ROWS OF ACTOR TABLE AFTER INSERTING NEW ROW")
print(tabulate(result,headers))
print()

new_id = result[0][0]

# UPDATE NEW ROW 
update_query = (
    "UPDATE actor "
    "SET first_name = 'KHANH2', last_name = 'BUI2' "
    "WHERE actor_id = %s"
)
cursor.execute(update_query,(new_id,))
db.commit()

# CHECK LAST 5 ROWS
check_query_2 = (
    "SELECT * FROM actor "
    "ORDER BY actor_id DESC "
    "LIMIT 5 "
)
cursor.execute(check_query_2)
result = cursor.fetchall()
headers = [i[0] for i in cursor.description]
print("LAST 5 ROWS OF ACTOR TABLE AFTER UPDATING NEW ROW")
print(tabulate(result,headers))
print()



# DELETE NEW ROW
delete_query = (
    "DELETE FROM actor "
    "WHERE actor_id = %s"
)

cursor.execute(delete_query,(new_id,))
db.commit()

# CHECK LAST 5 ROWS
check_query_3 = (
    "SELECT * FROM actor "
    "ORDER BY actor_id DESC "
    "LIMIT 5 "
)
cursor.execute(check_query_3)
result = cursor.fetchall()
headers = [i[0] for i in cursor.description]
print("LAST 5 ROWS OF ACTOR TABLE AFTER DELETING NEW ROW")
print(tabulate(result,headers))

LAST 5 ROWS OF ACTOR TABLE AFTER INSERTING NEW ROW
  actor_id  first_name    last_name    last_update
----------  ------------  -----------  -------------------
       201  KHANH         BUI          2024-04-07 17:29:14
       200  THORA         TEMPLE       2006-02-15 04:34:33
       199  JULIA         FAWCETT      2006-02-15 04:34:33
       198  MARY          KEITEL       2006-02-15 04:34:33
       197  REESE         WEST         2006-02-15 04:34:33

LAST 5 ROWS OF ACTOR TABLE AFTER UPDATING NEW ROW
  actor_id  first_name    last_name    last_update
----------  ------------  -----------  -------------------
       201  KHANH2        BUI2         2024-04-07 17:29:14
       200  THORA         TEMPLE       2006-02-15 04:34:33
       199  JULIA         FAWCETT      2006-02-15 04:34:33
       198  MARY          KEITEL       2006-02-15 04:34:33
       197  REESE         WEST         2006-02-15 04:34:33

LAST 5 ROWS OF ACTOR TABLE AFTER DELETING NEW ROW
  actor_id  first_name    last_name  

## 6. Benchmark similar query

In [11]:
def benchmark(query):
    import time
    start = time.time()
    for i in range(200):
        cursor.execute(query)
        result = cursor.fetchall()
    end = time.time()
    return end-start

In [12]:
query_1 = (
    "SELECT a.first_name, a.last_name, COUNT(f.film_id) film_count "
    "FROM actor a "
    "JOIN film_actor fa ON a.actor_id = fa.actor_id "
    "JOIN film f ON f.film_id = fa.film_id "
    "GROUP BY a.actor_id, a.first_name, a.last_name "
    "LIMIT 10"
) # LIST OF ACTORS AND NUMBER OF FILMS THEY STARRED IN

query_2 = (
    "WITH "
        "actor_film_count AS "
        "( "
        "SELECT a.actor_id, COUNT(f.film_id) film_count "
        "FROM actor a "
        "JOIN film_actor fa ON a.actor_id = fa.actor_id "
        "JOIN film f ON f.film_id = fa.film_id "
        "GROUP BY a.actor_id "
        ") "
    "SELECT a.first_name, a.last_name, afc.film_count "
    "FROM actor a "
    "JOIN actor_film_count afc ON a.actor_id = afc.actor_id "
    "LIMIT 10 "
) # LIST OF ACTORS AND NUMBER OF FILMS THEY STARRED IN

print(f"Query 1: {benchmark(query_1)}")
print(f"Query 2: {benchmark(query_2)}")

Query 1: 1.8903088569641113
Query 2: 1.2115652561187744


## 7. Optimize by adding index and benchmark again

In [13]:
# add index
alter_query = (
    "ALTER TABLE actor ADD INDEX actor_name (actor_id,first_name,last_name) "
)
cursor.execute(alter_query)

query_1 = (
    "SELECT a.first_name, a.last_name, COUNT(f.film_id) film_count "
    "FROM actor a "
    "JOIN film_actor fa ON a.actor_id = fa.actor_id "
    "JOIN film f ON f.film_id = fa.film_id "
    "GROUP BY a.actor_id, a.first_name, a.last_name "
    "LIMIT 10"
) # LIST OF ACTORS AND NUMBER OF FILMS THEY STARRED IN

query_2 = (
    "WITH "
        "actor_film_count AS "
        "( "
        "SELECT a.actor_id, COUNT(f.film_id) film_count "
        "FROM actor a "
        "JOIN film_actor fa ON a.actor_id = fa.actor_id "
        "JOIN film f ON f.film_id = fa.film_id "
        "GROUP BY a.actor_id "
        ") "
    "SELECT a.first_name, a.last_name, afc.film_count "
    "FROM actor a "
    "JOIN actor_film_count afc ON a.actor_id = afc.actor_id "
    "LIMIT 10 "
) # LIST OF ACTORS AND NUMBER OF FILMS THEY STARRED IN

print(f"Query 1: {benchmark(query_1)}")
print(f"Query 2: {benchmark(query_2)}")

# drop index
alter_query = (
    "DROP INDEX actor_name ON actor "
)
cursor.execute(alter_query)

Query 1: 1.8628051280975342
Query 2: 1.140791654586792


## 8. Close connection

In [14]:
db.close()